In [23]:
#import packages here

import spotipy
import spotipy.util as util
import json
import types
import pandas as pd
import json
from pandas.io.json import json_normalize  

In [24]:
#auth with spotipy and spotify app auth info here
#see documentation for spotipy/spotify setup: 
#https://spotipy.readthedocs.io/en/latest/?highlight=current_user_recently_played

token = util.prompt_for_user_token(
        username="psamrai7",
        scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing",
        client_id="06fbc86c86fb476fa9154ebbcea09a2a",
        client_secret="1ad66a32c99944ffa9728e38629c5ebb",
        redirect_uri="http://localhost/")

In [29]:
#perform data pull here

#useful forum post due to lack of updates to spotipy package
#https://stackoverflow.com/questions/50276854/how-to-retrieve-a-listening-history-object-by-spotipy

def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

spotify = spotipy.Spotify(auth=token)

spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)
recent_listening = spotify.current_user_recently_played(limit=50)

#raw data output optional
#out_file = open("recent_listening.json","w")
#out_file.write(json.dumps(recent_listening, sort_keys=True, indent=2))
#out_file.close()

In [30]:
#wrangle data to get approp structured lists here

#documentation: https://developer.spotify.com/documentation/web-api/reference/player/get-recently-played/
#assess json structure further and see if can remove elements of interest
#general structure:
# print(json.dumps(recent_listening['items'],indent=2))
# all artists involved in song:
# print(json.dumps(recent_listening['items'][0]['track']['artists'],indent=2))
# number of artists involved in song:
# sometimes there are multiple
# print(len(recent_listening['items'][0]['track']['artists']))
# each artist involved in song:
# print(json.dumps(recent_listening['items'][0]['track']['artists'][0]['name'],indent=2))
# each song:
# print(json.dumps(recent_listening['items'][0]['track']['name'],indent=2))

#extract artist(s) name
play_times = []
song_names = []
song_uri = []
song_popularity = []
song_duration = []
song_explicit = []

number_artists = []
primary_artist = []
second_artist = []
third_artist = []
forth_artist = []
fifth_artist = []

for i in range(len(recent_listening['items'])):
    play_times.append(recent_listening['items'][i]['played_at']) #play time (when track was played)
    song_names.append(recent_listening['items'][i]['track']['name']) #get each song name
    song_uri.append(recent_listening['items'][i]['track']['uri']) #get each song uri (unique record id for spotify)
    song_popularity.append(recent_listening['items'][i]['track']['popularity'])#get song popularity rank
    song_duration.append(recent_listening['items'][i]['track']['duration_ms']) #get duration time in ms
    song_explicit.append(recent_listening['items'][i]['track']['explicit']) #get explicit content bool 
                                
    #each primary artist involved in song
    primary_artist.append(recent_listening['items'][i]['track']['artists'][0]['name'])
    #second, third, forth, or fifth artist involved (if exist)
    
    n_artists = len(recent_listening['items'][i]['track']['artists'])
    number_artists.append(n_artists)
    
    if n_artists>1: 
        second_artist.append(recent_listening['items'][i]['track']['artists'][1]['name']) #get second artist
    else: 
        second_artist.append("")
    if n_artists>2: 
        third_artist.append(recent_listening['items'][i]['track']['artists'][2]['name']) #get third artist
    else: 
        third_artist.append("")    
    if n_artists>3: 
        forth_artist.append(recent_listening['items'][i]['track']['artists'][3]['name']) #get forth artist
    else: 
        forth_artist.append("")    
    if n_artists>4: 
        fifth_artist.append(recent_listening['items'][i]['track']['artists'][4]['name']) #get fifth artist
    else: 
        fifth_artist.append("")    

In [31]:
#quick check here

print("There were {} songs, {} primary artists, and {} play times captured.".format(len(song_names),
                                                                        len(primary_artist),
                                                                        len(play_times)))

There were 50 songs, 50 primary artists, and 50 play times captured.


In [32]:
#create dataframe and output it to csv here

user_songlisten_data = pd.DataFrame(
{"play_times" : play_times,  
"song_names" : song_names,  
"song_uri" : song_uri,  
"song_popularity" : song_popularity,  
"song_duration" : song_duration,  
"song_explicit" : song_explicit,  
"number_artists" : number_artists,
"primary_artist" : primary_artist,  
"second_artist" : second_artist,  
"third_artist" : third_artist,  
"forth_artist" : forth_artist,  
"fifth_artist" : fifth_artist
})
user_songlisten_data
print(type(user_songlisten_data))

#output file
user_songlisten_data.to_csv("user1_songlisten_data.csv", encoding='utf-8', index=False)

<class 'pandas.core.frame.DataFrame'>
